In [1]:

import pandas as pd
import numpy as np

# Load raw dataset
df = pd.read_csv("../data/data_raw/Loan_default.csv")
df.head()


,LoanID,Age,Income,LoanAmount,CreditScore,MonthsEmployed,NumCreditLines,InterestRate,LoanTerm,DTIRatio,Education,EmploymentType,MaritalStatus,HasMortgage,HasDependents,LoanPurpose,HasCoSigner,Default
0,I38PQUQS96,56,85994,50587,520,80,4,15.23,36,0.44,Bachelor's,Full-time,Divorced,Yes,Yes,Other,Yes,0
1,HPSK72WA7R,69,50432,124440,458,15,1,4.81,60,0.68,Master's,Full-time,Married,No,No,Other,Yes,0
2,C1OZ6DPJ8Y,46,84208,129188,451,26,3,21.17,24,0.31,Master's,Unemployed,Divorced,Yes,Yes,Auto,No,1
3,V2KKSFM3UN,32,31713,44799,743,0,3,7.07,24,0.23,High School,Full-time,Married,No,No,Business,No,0
4,EY08JDHTZP,60,20437,9139,633,8,4,6.51,48,0.73,Bachelor's,Unemployed,Divorced,No,Yes,Auto,No,0


In [2]:

print("Shape:", df.shape)
print("Columns:", df.columns.tolist())


Shape: (255347, 18)
Columns: ['LoanID', 'Age', 'Income', 'LoanAmount', 'CreditScore', 'MonthsEmployed', 'NumCreditLines', 'InterestRate', 'LoanTerm', 'DTIRatio', 'Education', 'EmploymentType', 'MaritalStatus', 'HasMortgage', 'HasDependents', 'LoanPurpose', 'HasCoSigner', 'Default']


# Quick EDA (dtypes, missingness, target distribution)

In [3]:

# Cell B — quick EDA
target = "Default"

# dtypes and basic describe
print(df.dtypes)
display(df.describe(include="all").T)

# Missingness (counts and percent)
missing = df.isnull().sum().sort_values(ascending=False)
missing_pct = (df.isnull().mean().sort_values(ascending=False) * 100).round(2)
missing_df = pd.concat([missing, missing_pct], axis=1, keys=['missing_count', 'missing_pct'])
display(missing_df[missing_df['missing_count']>0].head(20))

# Target counts & normalized (if target exists)
if target in df.columns:
    print("Target distribution (counts):")
    print(df[target].value_counts())
    print("\nTarget distribution (proportions):")
    print(df[target].value_counts(normalize=True).round(3))
else:
    print(f"⚠️ Target column '{target}' not found.")


LoanID             object
Age                 int64
Income              int64
LoanAmount          int64
CreditScore         int64
MonthsEmployed      int64
NumCreditLines      int64
InterestRate      float64
LoanTerm            int64
DTIRatio          float64
Education          object
EmploymentType     object
MaritalStatus      object
HasMortgage        object
HasDependents      object
LoanPurpose        object
HasCoSigner        object
Default             int64
dtype: object


,count,unique,top,freq,mean,std,min,25%,50%,75%,max
LoanID,255347,255347,I38PQUQS96,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Age,255347.0,NaN,NaN,NaN,43.498306,14.990258,18.0,31.0,43.0,56.0,69.0
Income,255347.0,NaN,NaN,NaN,82499.304597,38963.013729,15000.0,48825.5,82466.0,116219.0,149999.0
LoanAmount,255347.0,NaN,NaN,NaN,127578.865512,70840.706142,5000.0,66156.0,127556.0,188985.0,249999.0
CreditScore,255347.0,NaN,NaN,NaN,574.264346,158.903867,300.0,437.0,574.0,712.0,849.0
MonthsEmployed,255347.0,NaN,NaN,NaN,59.541976,34.643376,0.0,30.0,60.0,90.0,119.0
NumCreditLines,255347.0,NaN,NaN,NaN,2.501036,1.117018,1.0,2.0,2.0,3.0,4.0
InterestRate,255347.0,NaN,NaN,NaN,13.492773,6.636443,2.0,7.77,13.46,19.25,25.0
LoanTerm,255347.0,NaN,NaN,NaN,36.025894,16.96933,12.0,24.0,36.0,48.0,60.0
DTIRatio,255347.0,NaN,NaN,NaN,0.500212,0.230917,0.1,0.3,0.5,0.7,0.9


,missing_count,missing_pct


Target distribution (counts):
Default
0    225694
1     29653
Name: count, dtype: int64

Target distribution (proportions):
Default
0    0.884
1    0.116
Name: proportion, dtype: float64


# Group columns by type & show unique counts / cardinality

In [4]:

# Cell C — column grouping + cardinality
target = "Default"

numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
categorical_cols = df.select_dtypes(include=["object", "category", "bool"]).columns.tolist()

# Remove target from numeric_cols if present
if target in numeric_cols:
    numeric_cols.remove(target)

print("Numeric columns:", numeric_cols)
print("Categorical columns:", categorical_cols)

# Show unique values (cardinality) for categorical columns to decide encoding
for col in categorical_cols:
    print(f"{col}: nunique={df[col].nunique()} | top values: {df[col].value_counts().head(5).to_dict()}")


Numeric columns: ['Age', 'Income', 'LoanAmount', 'CreditScore', 'MonthsEmployed', 'NumCreditLines', 'InterestRate', 'LoanTerm', 'DTIRatio']
Categorical columns: ['LoanID', 'Education', 'EmploymentType', 'MaritalStatus', 'HasMortgage', 'HasDependents', 'LoanPurpose', 'HasCoSigner']
LoanID: nunique=255347 | top values: {'I38PQUQS96': 1, 'HPSK72WA7R': 1, 'C1OZ6DPJ8Y': 1, 'V2KKSFM3UN': 1, 'EY08JDHTZP': 1}
Education: nunique=4 | top values: {"Bachelor's": 64366, 'High School': 63903, "Master's": 63541, 'PhD': 63537}
EmploymentType: nunique=4 | top values: {'Part-time': 64161, 'Unemployed': 63824, 'Self-employed': 63706, 'Full-time': 63656}
MaritalStatus: nunique=3 | top values: {'Married': 85302, 'Divorced': 85033, 'Single': 85012}
HasMortgage: nunique=2 | top values: {'Yes': 127677, 'No': 127670}
HasDependents: nunique=2 | top values: {'Yes': 127742, 'No': 127605}
LoanPurpose: nunique=5 | top values: {'Business': 51298, 'Home': 51286, 'Education': 51005, 'Other': 50914, 'Auto': 50844}
HasC

# Decide drops & small cleaning (IDs, suspect cols)

In [5]:

# Cell D — drop list & simple cleaning
# 1) Drop identifier-like columns
drop_cols = ["LoanID"]  # LoanID is a unique identifier → drop

# 2) If any columns have >40% missing, drop them (adjust threshold if needed)
missing_threshold = 0.40
high_missing = missing_df[missing_df['missing_pct'] > (missing_threshold * 100)].index.tolist()
print("High-missing columns (will drop):", high_missing)
drop_cols += [c for c in high_missing if c not in drop_cols]

# 3) Drop them from df copy for preprocessing
df_clean = df.drop(columns=drop_cols).copy()
print("Dropped columns:", drop_cols)
print("New shape:", df_clean.shape)


High-missing columns (will drop): []
Dropped columns: ['LoanID']
New shape: (255347, 17)


# Map binary categorical -> 0/1 (HasMortgage, HasDependents, HasCoSigner)

In [6]:
# Cell E — map Yes/No to 0/1 properly
binary_candidates = ["HasMortgage", "HasDependants", "HasCoSigner"]

binary_cols = [c for c in binary_candidates if c in df_clean.columns]
print("Binary columns detected:", binary_cols)

# Robust mapping dictionary (handles multiple cases)
map_yesno = {
    "YES": 1, "NO": 0,
    "Y": 1, "N": 0,
    "TRUE": 1, "FALSE": 0,
    True: 1, False: 0,
    1: 1, 0: 0
}

for c in binary_cols:
    df_clean[c] = (
        df_clean[c]
        .astype(str)
        .str.strip()
        .str.upper()        # standardize to YES/NO
        .map(map_yesno)     # now matches dictionary keys
    )

# Check result
print(df_clean[binary_cols].head())
print(df_clean[binary_cols].isna().sum())


Binary columns detected: ['HasMortgage', 'HasCoSigner']
   HasMortgage  HasCoSigner
0            1            1
1            0            1
2            1            0
3            0            0
4            0            0
HasMortgage    0
HasCoSigner    0
dtype: int64


# Recompute numeric/categorical lists & inspect

In [7]:

# Cell F — recompute groups after binary mapping
numeric_cols = df_clean.select_dtypes(include=[np.number]).columns.tolist()
if target in numeric_cols:
    numeric_cols.remove(target)

categorical_cols = df_clean.select_dtypes(include=["object", "category"]).columns.tolist()

print("Final numeric_cols:", numeric_cols)
print("Final categorical_cols:", categorical_cols)


Final numeric_cols: ['Age', 'Income', 'LoanAmount', 'CreditScore', 'MonthsEmployed', 'NumCreditLines', 'InterestRate', 'LoanTerm', 'DTIRatio', 'HasMortgage', 'HasCoSigner']
Final categorical_cols: ['Education', 'EmploymentType', 'MaritalStatus', 'HasDependents', 'LoanPurpose']


### Imputation & Transformation Rules

- **Numeric Features**
  - **Imputation**: Use **median** imputation.  
    - Reason: The median is robust to outliers (e.g., if one person has income = 10M while most are around 50K, the mean would be skewed).  
  - **Scaling**: Apply **StandardScaler** (mean=0, std=1).  
    - Reason: Many ML algorithms (Logistic Regression, SVM, Neural Nets) assume features are on a comparable scale.  
    - Scaling prevents large-value features (like Income) from dominating smaller-value ones (like NumCreditLines).

- **Categorical Features**
  - **Imputation**: Use **most_frequent (mode)** imputation.  
    - Reason: Missing categories can usually be replaced with the most common class without introducing noise.  
    - Alternative could be filling with "Unknown", but mode is cleaner when missingness is low.  
  - **Encoding**: Apply **OneHotEncoder(handle_unknown="ignore")**.  
    - Reason: Converts categories into binary vectors.  
    - `handle_unknown="ignore"` ensures that if a new category appears in test data, the model won’t crash.

- **Target Column**
  - Keep **Default** (0 = No Default, 1 = Default) untouched.  
  - Reason: It is already binary and clean.


# Build ColumnTransformer + classifier pipeline

In [8]:

# Cell H — pipeline building + fit example
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

# Drop problematic numeric columns with all-missing values if present
numeric_cols = [col for col in numeric_cols if col not in ["HasMortgage", "HasDependents", "HasCoSigner"]]

print("Using numeric:", numeric_cols)
print("Using categorical:", categorical_cols)

# Define pipelines
num_pipeline = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

cat_pipeline = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore", sparse_output=False))
])

preprocessor = ColumnTransformer(transformers=[
    ("num", num_pipeline, numeric_cols),
    ("cat", cat_pipeline, categorical_cols)
], remainder="drop")

# Make feature matrix X and target y
X = df_clean.drop(columns=[target])
y = df_clean[target].astype(int)  # ensure integer target

# Train/test split (stratify to keep class proportions)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Full pipeline with logistic regression (balanced classes)
clf = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("model", LogisticRegression(max_iter=500, class_weight="balanced", random_state=42))
])

# Fit
clf.fit(X_train, y_train)

# Predict + report
y_pred = clf.predict(X_test)
y_proba = clf.predict_proba(X_test)[:, 1]

print("Classification report:")
print(classification_report(y_test, y_pred))

print("Confusion matrix:")
print(confusion_matrix(y_test, y_pred))

print("ROC AUC:", round(roc_auc_score(y_test, y_proba), 4))


Using numeric: ['Age', 'Income', 'LoanAmount', 'CreditScore', 'MonthsEmployed', 'NumCreditLines', 'InterestRate', 'LoanTerm', 'DTIRatio']
Using categorical: ['Education', 'EmploymentType', 'MaritalStatus', 'HasDependents', 'LoanPurpose']
Classification report:
              precision    recall  f1-score   support

           0       0.94      0.67      0.78     45139
           1       0.22      0.70      0.33      5931

    accuracy                           0.67     51070
   macro avg       0.58      0.69      0.56     51070
weighted avg       0.86      0.67      0.73     51070

Confusion matrix:
[[30300 14839]
 [ 1775  4156]]
ROC AUC: 0.7498


In [9]:

# Cell F — Fix Target Column ("Default")

# 1. Inspect current target column
print("Unique values in 'Default':", df_clean["Default"].unique())
print(df_clean["Default"].value_counts())

# 2. If already 0/1, just rename for consistency
if set(df_clean["Default"].unique()).issubset({0, 1}):
    df_clean.rename(columns={"Default": "target"}, inplace=True)
    print("✅ Column 'Default' renamed to 'target' (already 0/1 encoding).")

# 3. If values are Yes/No, True/False, etc. → map to 0/1
else:
    df_clean["target"] = df_clean["Default"].map({
        "Yes": 1, "No": 0,
        "True": 1, "False": 0,
        1: 1, 0: 0
    })
    print("✅ Column 'Default' mapped to 0/1 and saved as 'target'.")

# 4. Verify distribution
print("\nTarget distribution:")
print(df_clean["target"].value_counts(normalize=True).round(3))


Unique values in 'Default': [0 1]
Default
0    225694
1     29653
Name: count, dtype: int64
✅ Column 'Default' renamed to 'target' (already 0/1 encoding).

Target distribution:
target
0    0.884
1    0.116
Name: proportion, dtype: float64


# Cell G — Categorical Encoding Plan

- **Low-cardinality categorical columns** (few unique values, e.g., `term`, `emp_length`, `home_ownership`) → OneHotEncode.  
- **Medium-cardinality columns** (e.g., `purpose`) → OneHotEncode if reasonable.  
- **High-cardinality columns** (e.g., `zip_code`, `addr_state`) → Drop or group into broader categories.  
- Final decision: only keep categorical features that are meaningful and won’t explode the feature space.


# Cell H — Imputation Plan

In [10]:
# Cell H — Imputation Plan

import numpy as np

# Numeric columns (excluding target)
numeric_cols = df_clean.select_dtypes(include=[np.number]).columns.tolist()
if "target" in numeric_cols:
    numeric_cols.remove("target")

# Categorical columns
categorical_cols = df_clean.select_dtypes(include=["object", "category"]).columns.tolist()

print("Numeric columns (median impute):", numeric_cols[:10], "... total:", len(numeric_cols))
print("Categorical columns (Unknown impute):", categorical_cols[:10], "... total:", len(categorical_cols))

# Document strategy (actual imputation will be inside sklearn Pipeline later)


Numeric columns (median impute): ['Age', 'Income', 'LoanAmount', 'CreditScore', 'MonthsEmployed', 'NumCreditLines', 'InterestRate', 'LoanTerm', 'DTIRatio', 'HasMortgage'] ... total: 11
Categorical columns (Unknown impute): ['Education', 'EmploymentType', 'MaritalStatus', 'HasDependents', 'LoanPurpose'] ... total: 5


# Cell I — Double Check Target Distribution

In [11]:
# Cell I — Target check
print("Target distribution after encoding:")
print(df_clean["target"].value_counts(normalize=True).round(3))


Target distribution after encoding:
target
0    0.884
1    0.116
Name: proportion, dtype: float64


# Cell J — Save Cleaned Dataset

In [12]:
# Cell J — Save cleaned dataset
out_path = "../data/data_processed/loan_clean.csv"
df_clean.to_csv(out_path, index=False)
print(f"✅ Saved cleaned dataset to {out_path}")


✅ Saved cleaned dataset to ../data/data_processed/loan_clean.csv
